In [ ]:
import numpy as np
import pandas as pd
import cv2
from PIL import Image
from tqdm import tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('drive/MyDrive/CS2/dataframe.csv')
df = df.drop(['Unnamed: 0'], axis=1)
df.head()


,filename,width,height,class,xmin,ymin,xmax,ymax
0,Czech_000512.jpg,600,600,D20,403,467,559,538
1,Czech_000512.jpg,600,600,D40,321,473,398,522
2,Czech_000646.jpg,600,600,D00,250,400,287,515
3,Czech_003379.jpg,600,600,D00,298,389,354,515
4,Czech_000809.jpg,600,600,D00,264,453,391,527


In [ ]:
len(df)

33104

In [ ]:
uniqueImages = open('uniqueImg.txt','w')

imgs = df['filename'].unique()
for im in tqdm(imgs):
  uniqueImages.write(str(im))
  uniqueImages.write('\n')
uniqueImages.close()

100%|██████████| 14075/14075 [00:00<00:00, 1081124.97it/s]


In [ ]:

def getFileCounts(folder):
    files = os.listdir(folder)
    count = 0
    for file in files:
        count += 1
    return count

def removeUnwantedFiles(fileList,folder):
    #this function removes the files in fileList from folder:
    
    with open(fileList, 'r') as f:
        for line in f:
            fileName = line.rstrip()
            print(fileName)
            path = folder + '/' + fileName
            os.remove(path)
        f.close()

def getFileName(folder):
    allImg = open('allImg.txt','w')
    files = os.listdir(folder)
    for file in files:
        allImg.write(str(file))
        allImg.write('\n')
    allImg.close()


def getOutliers(allFiles, uniqueFiles, folder):
    allImages = []
    imgToKeep = []
    imgToDel = []
    with open(allFiles, 'r') as f:
        for line in f:
            fileName = line.rstrip()
            allImages.append(fileName)
        f.close()
    
    with open(uniqueFiles, 'r') as f:
        for line in f:
            fileName = line.rstrip()
            imgToKeep.append(fileName)
        f.close()   

    for img in allImages:
        if img not in imgToKeep:
            imgToDel.append(img)
    print(len(imgToDel))

    fileDel = open('imgToDel.txt','w')
    files = os.listdir(folder)
    for img in imgToDel:
        fileDel.write(str(img))
        fileDel.write('\n')
    fileDel.close()

#converting csv format to yolo format:
def csv_to_yolo(df):
  dir = 'darknet/data/labels/'
  labelsUsed = []
  labelsRepeat = []
  #D10,D40,D00,D20,D44,D50
  classIndex = {'D10':0, 'D40':1, 'D00':2, 'D20':3, 'D44':4, 'D50':5}
  for i in range(len(df)):
    fName = df['filename'].values[i].split('.')[0]
    if fName not  in labelsUsed:
      label = open(dir + str(fName) + '.txt', 'w')
      class_ = classIndex[df['class'].values[i]]
      image_width = df['width'].values[i]
      image_height = df['height'].values[i]
      xmin = df['xmin'].values[i]
      ymin = df['ymin'].values[i]
      xmax = df['xmax'].values[i]
      ymax = df['ymax'].values[i]

      x_coord = (xmin + xmax) / 2 / image_width
      y_coord = (ymin + ymax) / 2 / image_height
      shape_width = (xmax - xmin) / image_width
      shape_height = (ymax - ymin) / image_height

      label.write(str(class_) + ' ' + str(x_coord) + ' ' + str(y_coord) + ' ' + str(shape_width) + ' ' + str(shape_height))
      label.write('\n')
      label.close()
      labelsUsed.append(fName)
    else:
      label = open(dir + str(fName) + '.txt', 'a')
      class_ = classIndex[df['class'].values[i]]
      image_width = df['width'].values[i]
      image_height = df['height'].values[i]
      xmin = df['xmin'].values[i]
      ymin = df['ymin'].values[i]
      xmax = df['xmax'].values[i]
      ymax = df['ymax'].values[i]

      x_coord = (xmin + xmax) / 2 / image_width
      y_coord = (ymin + ymax) / 2 / image_height
      shape_width = (xmax - xmin) / image_width
      shape_height = (ymax - ymin) / image_height

      label.write(str(class_) + ' ' + str(x_coord) + ' ' + str(y_coord) + ' ' + str(shape_width) + ' ' + str(shape_height))
      label.write('\n')
      #label.read()
      label.close()
      
  #return labelsUsed, labelsRepeat

def trainTestSplit(uniqueImg):
    #listing directories for all the images in a txt file for train and validation set: 
    listImages = []
    with open(uniqueImg, 'r') as f:
        for line in f:
            fileName = line.rstrip()
            listImages.append(fileName)
        f.close()
    #listImages = df['filename'].values
    totalImages = len(listImages)
    print(totalImages)
    trainImages = int(0.8*totalImages)
    valImages = totalImages - trainImages

    trainFile = open('train.txt','w')
    valFile = open('val.txt','w')

    dir = 'data/images/'
    for i in range(trainImages):
        trainFile.write(dir + listImages[i])
        trainFile.write('\n')

    for i in range(valImages):
        valFile.write(dir + listImages[trainImages+i])
        valFile.write('\n')

    trainFile.close()
    valFile.close()


df = pd.read_csv('dataframe.csv')
csv_to_yolo(df)
            
#trainTestSplit('uniqueImg.txt')
#numImages = getFileCounts('darknet/data/images')
#numLabels = getFileCounts('darknet/data/labels')
#print('Total images : ',numImages)
#print('Total labels : ',numLabels)
#getFileName('darknet/data/images')
#getOutliers('allImg.txt', 'uniqueImg.txt', 'darknet/data/images')
#removeUnwantedFiles('imgToDel.txt', 'darknet/data/images' )
#numImages = getFileCounts('darknet/data/images')
#print('Total images : ',numImages)


In [ ]:
df['class'].unique()

array(['D10', 'D40', 'D00', 'D20', 'D44', 'D50'], dtype=object)

In [ ]:
images = os.listdir('data/images')
print(len(images))

14075


In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0


In [ ]:
!unzip "/content/drive/My Drive/CS2/darknet.zip"

Streaming output truncated to the last 5000 lines.
  inflating: darknet/data/labels/Japan_006380.txt  
  inflating: darknet/data/labels/Japan_006381.txt  
  inflating: darknet/data/labels/Japan_006384.txt  
  inflating: darknet/data/labels/Japan_006385.txt  
  inflating: darknet/data/labels/Japan_006386.txt  
  inflating: darknet/data/labels/Japan_006387.txt  
  inflating: darknet/data/labels/Japan_006388.txt  
  inflating: darknet/data/labels/Japan_006390.txt  
  inflating: darknet/data/labels/Japan_006391.txt  
  inflating: darknet/data/labels/Japan_006392.txt  
  inflating: darknet/data/labels/Japan_006393.txt  
  inflating: darknet/data/labels/Japan_006394.txt  
  inflating: darknet/data/labels/Japan_006395.txt  
  inflating: darknet/data/labels/Japan_006396.txt  
  inflating: darknet/data/labels/Japan_006398.txt  
  inflating: darknet/data/labels/Japan_006400.txt  
  inflating: darknet/data/labels/Japan_006402.txt  
  inflating: darknet/data/labels/Japan_006404.txt  
  inflating: 

In [ ]:

%cd /content/darknet 
!make
!chmod +x ./darknet

/content/darknet
mkdir -p obj
mkdir -p backup
mkdir -p results
gcc -Iinclude/ -Isrc/ -DOPENCV `pkg-config --cflags opencv`  -DGPU -I/usr/local/cuda/include/ -DCUDNN  -Wall -Wno-unused-result -Wno-unknown-pragmas -Wfatal-errors -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -c ./src/gemm.c -o obj/gemm.o
./src/gemm.c: In function ‘time_gpu’:
./src/gemm.c:232:9: warning: ‘cudaThreadSynchronize’ is deprecated [-Wdeprecated-declarations]
         cudaThreadSynchronize();
         ^~~~~~~~~~~~~~~~~~~~~
In file included from /usr/local/cuda/include/cuda_runtime.h:96:0,
                 from include/darknet.h:11,
                 from ./src/utils.h:5,
                 from ./src/gemm.c:2:
/usr/local/cuda/include/cuda_runtime_api.h:967:57: note: declared here
 extern __CUDA_DEPRECATED __host__ cudaError_t CUDARTAPI cudaThreadSynchronize(void);
                                                         ^~~~~~~~~~~~~~~~~~~~~
gcc -Iinclude/ -Isrc/ -DOPENCV `pkg-config --cflags opencv`  -DGPU -I/usr/local/cuda/

In [ ]:
!rm /content/darknet/backup -r
!ln -s /content/drive/'My Drive'/CS2/YOLOv3_weight/backup /content/darknet


In [ ]:
!sudo apt install dos2unix

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  dos2unix
0 upgraded, 1 newly installed, 0 to remove and 39 not upgraded.
Need to get 351 kB of archives.
After this operation, 1,267 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 dos2unix amd64 7.3.4-3 [351 kB]
Fetched 351 kB in 1s (384 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package dos2unix.
(Reading database ... 160772 files and directories curren

In [ ]:
!dos2unix ./data/train.txt
!dos2unix ./data/val.txt
!dos2unix ./data/yolo.data
!dos2unix ./data/yolo.names
!dos2unix ./cfg/yolov3_custom_train.cfg

dos2unix: converting file ./data/train.txt to Unix format...
dos2unix: converting file ./data/val.txt to Unix format...
dos2unix: converting file ./data/yolo.data to Unix format...
dos2unix: converting file ./data/yolo.names to Unix format...
dos2unix: converting file ./cfg/yolov3_custom_train.cfg to Unix format...


In [ ]:
%cd /content/darknet
!./darknet detector train data/yolo.data cfg/yolov3_custom_train.cfg darknet53.conv.74

Streaming output truncated to the last 5000 lines.
Region 106 Avg IOU: -nan, Class: -nan, Obj: -nan, No Obj: 0.000003, .5R: -nan, .75R: -nan,  count: 0
Region 82 Avg IOU: -nan, Class: -nan, Obj: -nan, No Obj: 0.000045, .5R: -nan, .75R: -nan,  count: 0
Region 94 Avg IOU: -nan, Class: -nan, Obj: -nan, No Obj: 0.000017, .5R: -nan, .75R: -nan,  count: 0
Region 106 Avg IOU: -nan, Class: -nan, Obj: -nan, No Obj: 0.000001, .5R: -nan, .75R: -nan,  count: 0
Region 82 Avg IOU: -nan, Class: -nan, Obj: -nan, No Obj: 0.000102, .5R: -nan, .75R: -nan,  count: 0
Region 94 Avg IOU: -nan, Class: -nan, Obj: -nan, No Obj: 0.000010, .5R: -nan, .75R: -nan,  count: 0
Region 106 Avg IOU: -nan, Class: -nan, Obj: -nan, No Obj: 0.000003, .5R: -nan, .75R: -nan,  count: 0
Region 82 Avg IOU: -nan, Class: -nan, Obj: -nan, No Obj: 0.000012, .5R: -nan, .75R: -nan,  count: 0
Region 94 Avg IOU: -nan, Class: -nan, Obj: -nan, No Obj: 0.000003, .5R: -nan, .75R: -nan,  count: 0
Region 106 Avg IOU: -nan, Class: -nan, Obj: -n